In [1]:
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import moscot
from moscot.problems.time import TemporalProblem
import moscot.plotting as mpl
import pandas as pd
import os
import muon
from ott.geometry import pointcloud
from sklearn.preprocessing import StandardScaler
import networkx as nx
import itertools
import anndata
from mudata import MuData
import jax.numpy as jnp
from typing import Dict, Tuple
from ott import tools
from tqdm import tqdm
import jax
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=200)
                         
import mplscience

mplscience.available_styles()
mplscience.set_style(reset_current=True)
plt.rcParams['legend.scatterpoints'] = 1 

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/icb/dominik.klein/mambaforge/envs/

['default', 'despine']


In [2]:
output_dir = "/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/stability_analysis"

In [3]:
mudata = muon.read("/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/mudata_with_annotation_all.h5mu")

endocrine_celltypes = [
    "Ngn3 low",
    "Ngn3 high",
    "Ngn3 high cycling",
    "Fev+",
    "Fev+ Alpha",
    "Fev+ Beta",
    "Fev+ Delta",
    "Eps. progenitors",
    "Alpha",
    "Beta",
    "Delta",
    "Epsilon"
]

mudata = mudata[mudata.obs["cell_type"].isin(endocrine_celltypes)].copy()

In [4]:
mudata.obsm

MuAxisArrays with keys: X_MultiVI, X_umap, atac, rna

In [5]:
RNA_emb = [None, "X_pca", "X_scVI"]
ATAC_emb = [None, "X_lsi", "X_poissonvi"]



In [6]:
EMBEDDINGS = list(itertools.product(RNA_emb, ATAC_emb)) + ["X_MultiVI"]

In [7]:
COSTS = [("sq_euclidean", None), ("cosine", None), ("geodesic", 5), ("geodesic", 10), ("geodesic", 30)]

In [8]:
EMB = "embedding"

In [9]:
def adapt_time(x):
        if x["stage"]=="E14.5":
            return 14.5
        if x["stage"]=="E15.5":
            return 15.5
        if x["stage"]=="E16.5":
            return 16.5
        raise ValueError
    
def create_adata(mudata: MuData, embedding: str) -> anndata.AnnData:

    adata = mudata["rna"]
    adata.obs["cell_type_refined"] = mudata.obs["cell_type_refined"]
    adata.obs['time'] = adata.obs.apply(adapt_time, axis=1).astype("category")
    if embedding == "X_MultiVI":
        adata.obsm[EMB] = mudata.obsm[embedding].copy()
    elif len(embedding)==2:
        rna_embedding, atac_embedding = embedding
        if rna_embedding is not None:
            rna_emb = mudata["rna"].obsm[rna_embedding]
            rna_emb_scaled = StandardScaler().fit_transform(rna_emb)
        if atac_embedding is not None:
            atac_emb = mudata["atac"].obsm[atac_embedding]
            atac_emb_scaled = StandardScaler().fit_transform(atac_emb)
        if rna_embedding is not None and atac_embedding is not None:
            emb = np.concatenate((rna_emb, atac_emb), axis=1)
        elif rna_embedding is None and atac_embedding is not None:
            emb = atac_emb
        elif rna_embedding is not None and atac_embedding is None:
            emb = rna_emb
        else:
            raise NotImplementedError
    
        adata.obsm[EMB] = emb
        return adata
    raise NotImplementedError


def create_graphs(adata: anndata.AnnData, n_neighbors: int) -> Dict[Tuple, pd.DataFrame]:
    dfs = {}
    batch_column = "time"
    unique_batches = [14.5, 15.5, 16.5]
    for i in range(len(unique_batches) - 1):
        batch1 = unique_batches[i]
        batch2 = unique_batches[i + 1]
    
        indices = np.where((adata.obs[batch_column] == batch1) | (adata.obs[batch_column] == batch2))[0]
        adata_subset = adata[indices]
        sc.pp.neighbors(adata_subset, use_rep=EMB, n_neighbors=n_neighbors)
        G = nx.from_numpy_array(adata_subset.obsp["connectivities"].A)
        assert nx.is_connected(G)
    
        dfs[(batch1, batch2)] = (
            pd.DataFrame(
                index=adata_subset.obs_names,
                columns=adata_subset.obs_names,
                data=adata_subset.obsp["connectivities"].A.astype("float"),
            )
        )
    return dfs

In [10]:
cm = jnp.ones((144, 144)) - jnp.eye(144)

def compute_metrics(df_reference: jax.Array, df: pd.DataFrame, emb: str, cost_0: str, cost_1: str) -> pd.DataFrame:
    
    sink_div = tools.sinkhorn_divergence.sinkhorn_divergence(geometry.Geometry, cost_matrix=(cm,cm,cm), a=df_reference.values.flatten(), b=df.values.flatten(), epsilon=1e-3).divergence
    eps_from_eps_prog = df.loc["Eps. progenitors", "Epsilon"]
    delta_from_fev_delta = df.loc["Delta", "Fev+ Delta"]
    fev_delta_from_eps_prog = df.loc["Eps. progenitors", "Fev+ Delta"]
    eps_from_fev_delta = df.loc["Fev+ Delta", "Epsilon"]
    print(emb, cost_0, cost_1, sink_div)
    print(eps_from_eps_prog, delta_from_fev_delta, fev_delta_from_eps_prog, eps_from_fev_delta)
    data = [[str(emb), str(cost_0), str(cost_1), sink_div, eps_from_eps_prog, delta_from_fev_delta, fev_delta_from_eps_prog, eps_from_fev_delta]]
    
    return pd.DataFrame(data=data, columns=["emb", "cost_0", "cost_1", "sink_div", "eps_from_eps_prog", "delta_from_dev_delta", "fev_delta_from_eps_prog", "eps_from_fev_delta"])
                            
                                      

In [11]:
tp_reference = TemporalProblem.load("/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/plots/OT_encodrine_analysis/TemporalProblem.pkl")


In [12]:
metrics_early = pd.DataFrame(columns=["emb", "cost_0", "cost_1", "sink_div", "eps_from_eps_prog", "delta_from_dev_delta", "fev_delta_from_eps_prog", "eps_from_fev_delta"])
metrics_late= pd.DataFrame(columns=["emb", "cost_0", "cost_1", "sink_div", "eps_from_eps_prog", "delta_from_dev_delta", "fev_delta_from_eps_prog", "eps_from_fev_delta"])

In [ ]:
order_cell_types = list(adata.obs["cell_type"].cat.categories)

In [ ]:
reference_tmap_early = tp_reference.cell_transition(14.5, 15.5, {"cell_type": order_cell_types}, {"cell_type": order_cell_types}, forward=False)
reference_tmap_late = tp_reference.cell_transition(15.5, 16.5, {"cell_type": order_cell_types}, {"cell_type": order_cell_types}, forward=False)
i=0
for emb in tqdm(EMBEDDINGS):
    if emb == (None, None):
        continue
    adata = create_adata(mudata, emb)
    for cost in COSTS:
    
        tp = TemporalProblem(adata)
        tp = tp.prepare("time", joint_attr=EMB, cost=cost[0] if cost[0]!="geodesic" else "sq_euclidean")

        if cost[0]=="geodesic":
            dfs = create_graphs(adata, cost[1])
            tp[14.5, 15.5].set_graph_xy((dfs[14.5, 15.5]).astype("float"), t=100.0)
            tp[15.5, 16.5].set_graph_xy((dfs[15.5, 16.5]).astype("float"), t=100.0)
        
        tp = tp.solve(lse_mode=False if cost[0]=="geodesic" else True, max_iterations=1e7, device='cpu')

        df_early = tp.cell_transition(14.5, 15.5, {"cell_type": order_cell_types}, {"cell_type": order_cell_types}, forward=False)
        df_late = tp.cell_transition(15.5, 16.5, {"cell_type": order_cell_types}, {"cell_type": order_cell_types}, forward=False)
        metrics_early = pd.concat((metrics_early, compute_metrics(reference_tmap_early, df_early, str(cost[0]), str(cost[1]), str(emb))))
        metrics_late = pd.concat((metrics_late, compute_metrics(reference_tmap_late, df_late, str(cost[0]), str(cost[1]), str(emb))))

        i+=1

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

INFO     Solving `2` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(5185, 1699)].                                  
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(4761, 5185)].                                  


/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

sq_euclidean None (None, 'X_lsi') 0.00049286336
0.07388929205126517 0.0011761732505029291 0.24813968657113114 0.03838017574168388


/tmp/ipykernel_487614/791395032.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_early = pd.concat((metrics_early, compute_metrics(reference_tmap_early, df_early, str(cost[0]), str(cost[1]), str(emb))))


...

geodesic 5 (None, 'X_lsi') nan
nan nan nan nan
geodesic 5 (None, 'X_lsi') nan
nan nan nan nan


/home/icb/dominik.klein/git_repos/moscot/src/moscot/problems/time/_mixins.py:986: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not (is_categorical_dtype(col) and is_numeric_dtype(col.cat.categories)):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: Future

INFO     Solving `2` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(5185, 1699)].                                  


/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


WARNING  Solver did not converge                                                                                   
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(4761, 5185)].                                  


In [ ]:
metrics_early.to_csv(os.path.join(output_dir, f"stability_metrics_early.csv"))
metrics_late.to_csv(os.path.join(output_dir, f"stability_metrics_late.csv"))

In [ ]:
cost[0], cost[1], str(emb)

In [ ]:
data = np.ones((8,1)).T

In [ ]:
pd.DataFrame(data=data, columns=["emb", "cost_0", "cost_1", "sink_div", "eps_from_eps_prog", "delta_from_dev_delta", "fev_delta_from_eps_prog", "eps_from_fev_delta"])

In [ ]:
out

In [ ]:
reference_tmap_early.shape

In [ ]:
reference_tmap_early.values.flatten()[:,None].shape

In [ ]:
df_early.values.flatten()[:,None].shape

In [ ]:
a = np.array([[1,2], [23,3]])
b = np.array([[13,1], [21,3]])

In [ ]:
sink_div = tools.sinkhorn_divergence.sinkhorn_divergence(pointcloud.PointCloud, x=a, y=b, epsilon=1e-1)
    

In [ ]:
sink_div = tools.sinkhorn_divergence.sinkhorn_divergence(pointcloud.PointCloud, pointcloud.PointCloud(reference_tmap_early.values.flatten()[:,None]), pointcloud.PointCloud(df_early.values.flatten()[:,None]), epsilon=1e-1)
    

In [ ]:
metrics_early.to_csv(os.path.join(output_dir, f"stability_metrics_early.csv"))
metrics_late.to_csv(os.path.join(output_dir, f"stability_metrics_late.csv"))